In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.clue import Clue, DeductiveSolver

total_num_turns = 0
playthroughs = 1

for _ in range(playthroughs):
    game = Clue(
        num_players=4,
        elements={
            "suspect": Clue.suspects[:6],
            "weapon": Clue.weapons[:6],
            "room": Clue.rooms[:9],
            # "motive": Clue.motives[:6],
            # "time": Clue.get_times("21:00", "03:00", "1h"),
        },
    )
    game.play(
        deductive_solver=DeductiveSolver(
            # note_cards_in_hand=False,
            # note_responses_to_suggestions=False,
            # note_cards_that_players_do_not_have=False,
            # check_unique_card_placement_constraints=False,
            # check_player_hand_size_constraints=False,
            check_solution_has_one_and_only_one_card_per_element=False,
            check_one_of_constraints=False,
            check_inverse_one_of_constraints=False,
            merge_and_check_disjoint_inverse_one_of_constraints=False,
            exhaustively_test_possible_assignments=False,
        ),
        check_if_deductive_solver_and_cp_solver_grids_match=False,
    )
    total_num_turns += game.num_turns

print(f"Average number of turns: {total_num_turns / playthroughs}")

Player 1's Hand: {'Mrs. Peacock', 'Mr. Green', 'Conservatory', 'Colonel Mustard'}
Player 2's Hand: {'Candlestick', 'Study', 'Hall', 'Miss Scarlet'}
Player 3's Hand: {'Knife', 'Dining Room', 'Mrs. White', 'Billiard Room', 'Kitchen'}
Player 4's Hand: {'Wrench', 'Library', 'Revolver', 'Lead Pipe', 'Ballroom'}
Solution: {'suspect': 'Professor Plum', 'weapon': 'Rope', 'room': 'Lounge'}
Player                   1  2  3  4
Element Card                       
Suspect Miss Scarlet     ✗  ✓  ✗  ✗
        Mr. Green        ✓  ✗  ✗  ✗
        Mrs. White       ✗  ✗  ✓  ✗
        Mrs. Peacock     ✓  ✗  ✗  ✗
        Colonel Mustard  ✓  ✗  ✗  ✗
        Professor Plum   ✗  ✗  ✗  ✗
Weapon  Candlestick      ✗  ✓  ✗  ✗
        Knife            ✗  ✗  ✓  ✗
        Lead Pipe        ✗  ✗  ✗  ✓
        Revolver         ✗  ✗  ✗  ✓
        Rope             ✗  ✗  ✗  ✗
        Wrench           ✗  ✗  ✗  ✓
Room    Hall             ✗  ✓  ✗  ✗
        Lounge           ✗  ✗  ✗  ✗
        Dining Room      ✗  ✗  ✓  ✗
    

In [4]:
players = [
    "Jacob",
    "Emily",
    "Michael",
    "Madison",
    "Joshua",
    "Emma",
    "Matthew",
    "Olivia",
    "Daniel",
    "Hannah",
    "Christopher",
    "Abigail",
    "Andrew",
    "Isabella",
    "Ethan",
    "Samantha",
    "Joseph",
    "Elizabeth",
    "William",
    "Ashley",
    "Anthony",
    "Alexis",
    "David",
    "Sarah",
    "Alexander",
    "Sophia",
    "Nicholas",
    "Alyssa",
    "Ryan",
    "Grace",
    "Tyler",
    "Ava",
    "James",
    "Taylor",
    "John",
    "Brianna",
    "Jonathan",
    "Lauren",
    "Noah",
    "Chloe",
    "Brandon",
    "Natalie",
    "Christian",
    "Kayla",
    "Dylan",
    "Jessica",
    "Samuel",
    "Anna",
    "Benjamin",
    "Victoria",
    "Nathan",
    "Mia",
    "Zachary",
    "Hailey",
    "Logan",
    "Sydney",
    "Justin",
    "Jasmine",
    "Gabriel",
    "Julia",
    "Jose",
    "Morgan",
    "Austin",
    "Destiny",
    "Kevin",
    "Rachel",
    "Elijah",
    "Ella",
    "Caleb",
    "Kaitlyn",
    "Robert",
    "Megan",
    "Thomas",
    "Katherine",
    "Jordan",
    "Savannah",
    "Cameron",
    "Jennifer",
    "Jack",
    "Alexandra",
    "Hunter",
    "Allison",
    "Jackson",
    "Haley",
    "Angel",
    "Maria",
    "Isaiah",
    "Kaylee",
    "Evan",
    "Lily",
    "Isaac",
    "Makayla",
    "Luke",
    "Brooke",
    "Mason",
    "Nicole",
    "Jayden",
    "Mackenzie",
    "Jason",
    "Addison",
    "Gavin",
    "Stephanie",
    "Aaron",
    "Lillian",
    "Connor",
    "Andrea",
    "Aiden",
    "Faith",
    "Aidan",
    "Zoe",
    "Kyle",
    "Kimberly",
    "Juan",
    "Madeline",
    "Charles",
    "Alexa",
    "Luis",
    "Katelyn",
    "Adam",
    "Gabriella",
    "Lucas",
    "Gabrielle",
    "Brian",
    "Trinity",
    "Eric",
    "Amanda",
    "Adrian",
    "Kylie",
    "Nathaniel",
    "Mary",
    "Sean",
    "Paige",
    "Alex",
    "Riley",
    "Carlos",
    "Leah",
    "Bryan",
    "Jenna",
    "Ian",
    "Sara",
    "Owen",
    "Rebecca",
    "Jesus",
    "Michelle",
    "Landon",
    "Sofia",
    "Julian",
    "Vanessa",
    "Chase",
    "Jordan",
    "Cole",
    "Angelina",
    "Diego",
    "Caroline",
    "Jeremiah",
    "Avery",
    "Steven",
    "Audrey",
    "Sebastian",
    "Evelyn",
    "Xavier",
    "Maya",
    "Timothy",
    "Claire",
    "Carter",
    "Autumn",
    "Wyatt",
    "Jocelyn",
    "Brayden",
    "Ariana",
    "Blake",
    "Nevaeh",
    "Hayden",
    "Arianna",
    "Devin",
    "Jada",
    "Cody",
    "Bailey",
    "Richard",
    "Brooklyn",
    "Seth",
    "Aaliyah",
    "Dominic",
    "Amber",
    "Jaden",
    "Isabel",
    "Antonio",
    "Mariah",
    "Miguel",
    "Danielle",
    "Liam",
    "Melanie",
    "Patrick",
    "Sierra",
    "Carson",
    "Erin",
    "Jesse",
    "Amelia",
    "Tristan",
    "Molly",
    "Alejandro",
    "Isabelle",
    "Henry",
    "Melissa",
    "Victor",
    "Madelyn",
    "Trevor",
    "Jacqueline",
    "Bryce",
    "Marissa",
    "Jake",
    "Angela",
    "Riley",
    "Shelby",
    "Colin",
    "Leslie",
    "Jared",
    "Katie",
    "Jeremy",
    "Jade",
    "Mark",
    "Catherine",
    "Caden",
    "Diana",
    "Garrett",
    "Aubrey",
    "Parker",
    "Mya",
    "Marcus",
    "Amy",
    "Vincent",
    "Briana",
    "Kaleb",
    "Sophie",
    "Kaden",
    "Gabriela",
    "Brady",
    "Breanna",
    "Colton",
    "Gianna",
    "Kenneth",
    "Kennedy",
    "Joel",
    "Gracie",
    "Oscar",
    "Peyton",
    "Josiah",
    "Adriana",
    "Jorge",
    "Christina",
    "Ashton",
    "Courtney",
    "Cooper",
    "Daniela",
    "Tanner",
    "Lydia",
    "Eduardo",
    "Kathryn",
    "Paul",
    "Valeria",
    "Edward",
    "Layla",
    "Ivan",
    "Alexandria",
    "Preston",
    "Natalia",
    "Maxwell",
    "Angel",
    "Alan",
    "Laura",
    "Levi",
    "Charlotte",
    "Stephen",
    "Margaret",
    "Grant",
    "Cheyenne",
    "Nicolas",
    "Naomi",
    "Dakota",
    "Miranda",
    "Omar",
    "Mikayla",
    "Alexis",
    "Kelsey",
    "George",
    "Payton",
    "Eli",
    "Ana",
    "Collin",
    "Alicia",
    "Spencer",
    "Jillian",
    "Gage",
    "Daisy",
    "Max",
    "Mckenzie",
    "Ricardo",
    "Ashlyn",
    "Cristian",
    "Sabrina",
    "Derek",
    "Caitlin",
    "Micah",
    "Summer",
    "Brody",
    "Ruby",
    "Francisco",
    "Valerie",
    "Nolan",
    "Rylee",
    "Ayden",
    "Skylar",
    "Dalton",
    "Lindsey",
    "Shane",
    "Kelly",
    "Peter",
    "Genesis",
    "Damian",
    "Zoey",
    "Jeffrey",
    "Eva",
    "Brendan",
    "Sadie",
    "Travis",
    "Alexia",
    "Fernando",
    "Cassidy",
    "Peyton",
    "Kylee",
    "Conner",
    "Kendall",
    "Andres",
    "Jordyn",
    "Javier",
    "Kate",
    "Giovanni",
    "Jayla",
    "Shawn",
    "Karen",
    "Braden",
    "Tiffany",
    "Jonah",
    "Cassandra",
    "Bradley",
    "Juliana",
    "Cesar",
    "Reagan",
    "Emmanuel",
    "Caitlyn",
    "Manuel",
    "Giselle",
    "Edgar",
    "Serenity",
    "Mario",
    "Alondra",
    "Erik",
    "Lucy",
    "Edwin",
    "Bianca",
    "Johnathan",
    "Kiara",
    "Devon",
    "Crystal",
    "Erick",
    "Erica",
    "Wesley",
    "Angelica",
    "Oliver",
    "Hope",
    "Trenton",
    "Chelsea",
    "Hector",
    "Alana",
    "Malachi",
    "Liliana",
    "Jalen",
    "Brittany",
    "Raymond",
    "Camila",
    "Gregory",
    "Makenzie",
    "Abraham",
    "Lilly",
    "Elias",
    "Veronica",
    "Leonardo",
    "Abby",
    "Sergio",
    "Jazmin",
    "Donovan",
    "Adrianna",
    "Colby",
    "Delaney",
    "Marco",
    "Karina",
    "Bryson",
    "Ellie",
    "Martin",
    "Jasmin",
]

In [5]:
import random
from typing import Optional

random.shuffle(players)


def turn(
    player: int, suggestion: list[str], responses: dict[int, Optional[str]]
) -> str:
    return f"{players[player]} asked if anyone had {' or '.join(f'\'{card}\'' for card in suggestion)}:\n{'\n'.join(f'- {players[responding_player]} showed {players[player]} {f"'{card}'" if player == game.winner or responding_player == game.winner else 'a card'}' if card is not None else f'- {players[responding_player]} did not have any of the cards' for responding_player, card in responses.items())}"


print(
    f"""
On a {random.choice(("cool", "warm"))} {random.choice(("spring", "autumn"))} {random.choice(("morning", "afternoon", "evening", "day"))} {", ".join(player for player in players[:game.num_players - 1])} and {players[game.num_players - 1]} sat down to play a {random.choice(("friendly", "competitive", "casual"))} {random.choice(("mystery", "deduction", "sleuthing"))} game.

They {random.choice(("assembled", "gathered"))} {len(game.elements)} {random.choice(("decks", "groups", "stacks"))} of cards, each for a {random.choice(("different", "separate"))} {random.choice(("category", "type"))} of {random.choice(("information", "data"))} composed of the following:

{"\n\n".join(f"{category.capitalize()}:\n{'\n'.join(f'- ' + card for card in cards)}" for category, cards in game.elements.items())}

After randomly (and blindly) choosing one card from each {random.choice(("deck", "group", "stack"))} and placing them in the {random.choice(("center", "middle"))} of the table facedown, they shuffled the remaining cards and dealt out the following to each player:

{"\n".join(f'- {players[player]}: {len(game.hands[player])} cards' + (f' ({", ".join(f"'{card}'" for card in game.hands[player])})' if player == game.winner else "") for player in range(game.num_players))}

The game proceeded as follows:

1. On their turn, a player asked about a set of exactly {len(game.elements)} cards, one from each of the game's categories. (Note: Players could ask about any cards, including those in their own hand.)
2. The player directed this question to the other players in clockwise order, starting with the player to their left.
3. If a player had one or more of the asked-about cards, they had to show one of those cards (of their choice) to the asking player privately. The turn then ended, and play passed to the next player.
4. If a player did not have any of the asked-about cards, they said so, and the question passed to the next player in clockwise order.
5. This continued until either:
    a) A player showed a card to the asking player, or
    b) All the queried players had stated they didn't have any of the asked-about cards.
6. After a player's turn ended (either by being shown a card or having all queried players pass), play moved to the next player in clockwise order.

Here is how the game played out:

{"\n\n".join(turn(player % game.num_players, suggestion, responses) for player, (suggestion, responses) in enumerate(game.history))}

{random.choice(("At this point,", "Then, on their turn,"))} {players[game.winner]} was able to correctly {random.choice(("deduce", "infer"))} the {random.choice(("solution", "answer"))} and win the game.

What were the facedown cards in the {random.choice(("center", "middle"))} of the table?
""".strip()
)
display("--------------------")
print("\n".join(f"{category.capitalize()}: {card}" for category, card in game.solution.items()))

On a cool autumn afternoon Jeffrey, Jenna, Rebecca and Brian sat down to play a competitive deduction game.

They gathered 3 stacks of cards, each for a different category of information composed of the following:

Suspect:
- Miss Scarlet
- Mr. Green
- Mrs. White
- Mrs. Peacock
- Colonel Mustard
- Professor Plum

Weapon:
- Candlestick
- Knife
- Lead Pipe
- Revolver
- Rope
- Wrench

Room:
- Hall
- Lounge
- Dining Room
- Kitchen
- Ballroom
- Conservatory
- Billiard Room
- Library
- Study

After randomly (and blindly) choosing one card from each deck and placing them in the middle of the table facedown, they shuffled the remaining cards and dealt out the following to each player:

- Jeffrey: 4 cards
- Jenna: 4 cards
- Rebecca: 5 cards ('Knife', 'Dining Room', 'Mrs. White', 'Billiard Room', 'Kitchen')
- Brian: 5 cards

The game proceeded as follows:

1. On their turn, a player asked about a set of exactly 3 cards, one from each of the game's categories. (Note: Players could ask about any c

'--------------------'

Suspect: Professor Plum
Weapon: Rope
Room: Lounge


In [ ]:
lines = """
Based on the game play, we know that:

1. Hannah showed 'Lounge' to Jacob.
2. Hannah showed 'Mr. Green' to Jacob.
3. Hannah showed a card to Emma (which wasn't 'Candlestick' or 'Hall').

This means that Hannah didn't show either 'Lounge' or 'Mr. Green' to Emma. So, the card Hannah showed Emma must be the 'Candlestick'. 

Jacob's initial two cards were 'Knife' and 'Dining Room'. He never showed 'Dining Room' to anyone, so it must have been the card in the middle. 

Jacob correctly guessed the solution, meaning the facedown cards in the middle of the table were:

- Suspect: Miss Scarlet
- Weapon: Lead Pipe
- Room: Hall

Since Hannah didn't show 'Lounge' or 'Mr. Green' to Emma, and Emma didn't have 'Candlestick', Miss Scarlet had to be the third card Hannah showed to Emma. This makes Emma the only player with a 'Candlestick', 'Mr. Green', and 'Lounge'. The cards in Emma's hand are:

- 'Mrs. White'
- 'Knife'

The cards in Hannah's hand are:

- 'Candlestick'
- 'Lead Pipe'

So, the complete setup was:

Facedown cards:

- Suspect: Miss Scarlet
- Weapon: Lead Pipe
- Room: Hall

Emma's hand:

- Mrs. White
- Knife

Jacob's hand:

- Knife (already determined)
- Undisclosed (but not the 'Dining Room'/middle card)

Hannah's hand:

- Candlestick (already determined)
- Undisclosed
""".strip().splitlines()

[print(f"[{no}] {line}") for no, line in enumerate(lines, 1)]

In [ ]:
game.solution

In [ ]:
import numpy as np

for i, hand in enumerate(game.hands):
    print(f"Player {i + 1} has {len(hand)} cards")

player = 3

for suggesting_player, (suggestion, responses) in enumerate(game.history):
    suggesting_player %= game.num_players
    for responding_player, card in responses.items():
        if card is not None:
            if player != suggesting_player and player != responding_player:
                print(
                    f"Player {responding_player + 1} has at least one of {suggestion}"
                )

from lib.clue import DeductiveSolver

player = 3

game.print_grid(DeductiveSolver().grid(game, player))

In [ ]:
"""
Player                   1  2  3  4
Element Card                       
Suspect Miss Scarlet        ✗  ✗  ✗
        Mr. Green        ✗  ✗  ✓  ✗
        Mrs. White       ✗  ✗  ✗  ✓
        Mrs. Peacock     ✗  ✓  ✗  ✗
        Colonel Mustard     ✗  ✗   
        Professor Plum   ✗  ✗  ✗  ✓
Weapon  Candlestick      ✗  ✗  ✗   
        Knife            ✗  ✗  ✓  ✗
        Lead Pipe        ✓  ✗  ✗  ✗
        Revolver         ✗  ✗  ✓  ✗
        Rope             ✗  ✓  ✗  ✗
        Wrench              ✗  ✗   
"""